In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim
import os
import csv
import random
import copy
import seaborn as sns
import matplotlib.pyplot as plt

seed = 2710
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
with open('data/realworld_mobiact.pkl', 'rb') as f:
    x, y, k = pickle.load(f)
with open('data/realworld_mobiact_fs.pkl', 'rb') as f:
    fs = pickle.load(f)

print(x.shape, y.shape, k.shape, fs.shape)

(22599, 3, 128) (22599,) (22599,) (22599,)


In [6]:
mask_rw = (k < 15)
mask_ma = (k >= 15)

x_rw, y_rw, k_rw, fs_rw = x[mask_rw], y[mask_rw], k[mask_rw], fs[mask_rw]
x_ma, y_ma, k_ma, fs_ma = x[mask_ma], y[mask_ma], k[mask_ma], fs[mask_ma]

print(x_rw.shape, y_rw.shape, k_rw.shape, fs_rw.shape)
print(x_ma.shape, y_ma.shape, k_ma.shape, fs_ma.shape)

(11783, 3, 128) (11783,) (11783,) (11783,)
(10816, 3, 128) (10816,) (10816,) (10816,)


In [7]:
x_ma_train, x_ma_test, y_ma_train, y_ma_test = train_test_split(x_ma, y_ma, test_size=0.2, random_state=seed, stratify=y_ma, shuffle=True)

print(x_ma_train.shape, y_ma_train.shape)
print(x_ma_test.shape, y_ma_test.shape)

(8652, 3, 128) (8652,)
(2164, 3, 128) (2164,)


In [ ]:
class TSTRClassifier(nn.Module):
    def __init__(self, num_timesteps=128, num_channels=3, num_classes=4):
        super(TSTRClassifier, self).__init__()

        self.conv1 = nn.Conv1d(num_channels, 16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.25)

        self.fc_shared = nn.Linear(num_timesteps * 8, 100)

        self.fc_class = nn.Linear(100, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.relu(self.fc_shared(x))

        # Final output for class prediction
        class_outputs = self.fc_class(x)
        return class_outputs


def train_model(model, train_loader, val_loader, optimizer, num_epochs=100, augment=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    loss_fn = nn.CrossEntropyLoss()

    loss_train = []
    loss_val = []
    accuracy_val = []
    best_model_state = None
    best_loss = np.inf
    best_accuracy = 0
    best_f1 = 0

    # Set up linear learning rate decay
    lambda_lr = lambda epoch: 1 - epoch / num_epochs
    scheduler = LambdaLR(optimizer, lr_lambda=lambda_lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            # if augment:
                # x_batch, _ = augment_batch(x_batch)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        total_loss /= len(train_loader)
        loss_train.append(total_loss)

        # Update learning rate
        scheduler.step()

        val_accuracy, val_loss, val_f1 = evaluate_model(model, val_loader)
        if val_loss < best_loss:
            best_epoch = epoch
            best_accuracy = val_accuracy
            best_f1 = val_f1
            best_loss = val_loss
            best_model_state = model.state_dict().copy()

        loss_val.append(val_loss)
        accuracy_val.append(val_accuracy)

        current_lr = scheduler.get_last_lr()[0]
        if (epoch+1) % 10 == 0:
            print(f"\tEpoch {epoch + 1}/{num_epochs} - Train loss: {total_loss:.4f} - Val accuracy: {val_accuracy:.4f} - Val loss: {val_loss:.4f} - Val F1: {val_f1:.4f} - LR: {current_lr:.2e}")

    print(f"\tBest epoch: {best_epoch + 1} - Best val accuracy: {best_accuracy:.4f} - Best val loss: {best_loss:.4f} - Best val F1: {best_f1:.4f}\n")

    # Load best model state
    model.load_state_dict(best_model_state)

    return model

In [ ]:
model = TSTRClassifier()

train_model()